In [12]:
%%capture
pip install plotly pandas statsmodels kaleido scipy nbformat jinja2

In [20]:
import os
import json
import pandas as pd

output_directory = "output-busybox"
base_file = "init_base.json"

In [63]:
with open(base_file) as json_data:
    d = json.load(json_data)

In [62]:
def get_latest_nonLinux(df):
    # assumes df_kconfig
    return df.loc[df['committer_date_unix'].idxmax()]

In [21]:
def read_dataframe(stage, dtype={}, usecols=None, file=None):
    if not file:
        file = 'output'
    df = pd.read_csv(f'{output_directory}/{stage}/{file}.csv', dtype=dtype, usecols=usecols)
    if 'committer_date_unix' in df:
        df['committer_date'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
    return df

df_kconfig = read_dataframe('kconfig')


In [ ]:
sloc = get_latest_nonLinux(df_kconfig)

np.float64(209492.0)

In [60]:
plot_column_mapping = {
    "features": "model-features",
    "total-features": "model-features",
    "model-count": "model-literals",
    "model-count-time": "model-time"
}

In [71]:
latestData = dict()

In [ ]:

def fill_latest_nonLinux():
    for bp in d:
        if bp == "linux":
            continue
        projects = d[bp]["projects"]
        plots = d[bp]["plots"]
        for p in projects:
            if p not in latestData:
                latestData[p] = dict()
            df_kconfig = read_dataframe('kconfig')
            latest_all = get_latest_nonLinux(df_kconfig[df_kconfig["system"] == p])
            for plot in plots:
                latest = latest_all[plot_column_mapping.get(plot, plot)]
                latestData[p][plot] = {
                    "title": "Total Features",
                    "currentValue": str(latest),
                    "cmpLastWeek": "",
                    "cmpLastMonth": "",
                    "desc": "Total Features: Information, Description etc.",
                }
fill_latest_nonLinux()
from pprint import pprint
pprint(latestData)

In [77]:
def fill_latest_linux():
    for bp in d:
        if bp != "linux":
            continue
        projects = d[bp]["projects"]
        plots = d[bp]["plots"]
        for p in projects:
            arch = p.rsplit("/")[-1]
            if arch == "all":
                continue
            if p not in latestData:
                latestData[p] = dict()
            df_kconfig = read_dataframe('kconfig')
            latest_all = get_latest_nonLinux(df_kconfig[df_kconfig["architecture"] == arch])
            for plot in plots:
                latest = latest_all[plot_column_mapping.get(plot, plot)]
                latestData[p][plot] = {
                    "title": "Total Features",
                    "currentValue": str(latest),
                    "cmpLastWeek": "",
                    "cmpLastMonth": "",
                    "desc": "Total Features: Information, Description etc.",
                }
fill_latest_linux()
# from pprint import pprint
# pprint(latestData)


ValueError: attempt to get argmax of an empty sequence